# Notebook 3: Dyadic Regression
#### Econometric Methods for Social Spillovers and Networks
#### University of St. Gallen, September 28th to October 6th, 2020
##### _Bryan S. Graham, UC - Berkeley, bgraham@econ.berkeley.edu_
<br>
<br>
This is the third of a series of iPython Jupyter notebooks designed to accompany a series of instructional lectures given (virtually!) at the St. Gallen University from September 28th and October 6th, 2020. The scripts below were written for Python 3.6. The Anaconda distribution of Python, available at https://www.continuum.io/downloads, comes bundled with most the scientific computing packages used in these notebooks.
<br>
<br>
For more information about the course please visit my webpage at http://bryangraham.github.io/econometrics/.

#### Code citation:
Graham, Bryan S. (2020). "Notebook 2: Dyadic Regression: St. Gallen University Econometric Methods for Social Spillovers and Networks Course Jupyter Notebook," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 20 September 2020)

## Dyadic regression and the gravity model of international trade

João Santos Silva and Silvana Tenreyro (2006, _Review of Economics and Statistics_) advocate the application of Poisson regression methods to the analysis of trade flows across countries. Their work has been influential in the empirical trade literature. Google Scholar calls their work a "classic paper" in economics (see this [blog post](https://scholar.googleblog.com/2017/06/classic-papers-articles-that-have-stood.html) for more information on Google Scholar "classic papers"). An open access copy of the paper can be found [here](https://www.mitpressjournals.org/doi/pdf/10.1162/rest.88.4.641). Replication data and additional information about the dataset used in the paper can be found [here](http://personal.lse.ac.uk/tenreyro/lgw.html).    

In this notebook we will use the Santos Silva and Tenreyro (2006) dataset to illustrate our results on estimation and inference of dyadic regression models. The dataset is located in the "data" directory of the Github repository. You'll need to download the data and adjust the **data** directory defined below to run this notebook on your computer.

**References**    

Santos Silva, J.M.C. and Tenreyro, Silvana. (2006). "The Log of Gravity", _Review of Economics and Statistics_ 88 (4): 641 - 658.

In [1]:
# Direct Python to plot all figures inline (i.e., not in a separate window)
%matplotlib inline

# Main scientific computing modules
# Load library dependencies
import time
import numpy as np
import scipy as sp
import pandas as pd
import itertools as it

# Import matplotlib 
import matplotlib.pyplot as plt

# networkx module for the analysis of network data
import networkx as nx

We will use the **dyadic_regression()** function in the _netrics_ package. This package has dependencies with the _ipt_ package. We therefore add the directories with these two codebases to our system path and then load them.

In [2]:
# Append location of ipt module base directory to system path
# NOTE: only required if permanent install not made (see comments above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/ipt/')
sys.path.append('/Users/bgraham/Dropbox/Sites/software/netrics/')

# Load ipt and netrics modules
import ipt as ipt
import netrics as netrics

Location of Santos Silva and Tenreyro (2006) replication data.

In [3]:
# Directory where graphics files will be saved (and where example data are stored)
data =     '/Users/bgraham/Dropbox/Teaching/Short_Courses/St_Gallen/2018/Data/'

Load dataset into a pandas dataframe. Set (dyadic) multi-index and sort. Print out first few lines of the dataframe.

In [4]:
LogOfGravity = pd.read_stata(data+"Log of Gravity.dta")
LogOfGravity.set_index(['s1_im', 's2_ex'], drop = False, inplace = True)          # Set dataframe multi-index
LogOfGravity.sort_index(level = ['s1_im', 's2_ex'], inplace = True)
LogOfGravity.head()

s1_im  s2_ex  border  ex_feenstra  im_feenstra  landl_im  \
s1_im s2_ex                                                             
2     3          2      3       0       130120       580080         0   
      6          2      6       0       160240       580080         0   
      8          2      8       0       330320       580080         0   
      11         2     11       0       710360       580080         0   
      12         2     12       0       550400       580080         0   

             landl_ex  trade      lyim       lyex  ...  imuis  exuis  imcacm  \
s1_im s2_ex                                        ...                         
2     3             0    237  6.735698   7.407237  ...      0      0       0   
      6             0      0  6.735698   6.500353  ...      0      0       0   
      8             0     20  6.735698   8.661679  ...      0      0       0   
      11            0      0  6.735698   9.833121  ...      0      0       0   
      12            1  12355  6.735698  10.222010  ...      0      0       0   

             excacm  imcaricom  excaricom  imopen  exopen  open_wto  \
s1_im s2_ex                                                           
2     3           0          0          0       0       0         1   
      6           0          0          0       0       0         0   
      8           0          0          0       0       0         0   
      11          0          0          0       0       1         1   
      12          0          0          0       0       1         1   

             comfrt_wto  
s1_im s2_ex              
2     3               0  
      6               0  
      8               0  
      11              0  
      12              0  

[5 rows x 63 columns]

The **trade** variable (*Y*) measures trade from exporter *i* to importer *j* in thousands of US dollars. Dividing by 1,000 scales the variable better for estimation. The set of covariates included in *W* correspond to those in used in Table 3 of Santos Silva and Tenreyro (2006, p. 650). 

In [5]:
Y = LogOfGravity['trade'].copy(deep=True)/1000
#W = LogOfGravity[['lyex', 'lyim', 'ldist']].copy(deep=True)
W = LogOfGravity[['lypex', 'lypim', 'lyex', 'lyim', 'ldist', 'border', 'comlang', 'colony', \
                  'landl_ex', 'landl_im', 'lremot_ex', 'lremot_im', 'comfrt_wto', 'open_wto']].copy(deep=True)
W['constant'] = 1

The **dyadic_regression()** function fits simple linear, logistic and poisson regression models to dyadic data by the method of (dyadic or pairwise) composite maximum likelihood estimation. The function can accommodate directed outcomes, in which case a total of N(N-1) directed outcomes are observed (with N the number of agents), as well as undirected outcomes, in which case 0.5N(N-1) outcomes are observed. The program does not, as of yet, accommodate the presence of missing outcomes.    

The function allows for three methods of standard error construction. All three of these methods are described in the forthcoming _Handbook of Econometrics_ chapter on networks by Graham (forthcoming). The first method assumes independence (conditional on covariates) across dyads. In the case of directed outcome data this does allow for dependence of $ Y_{ij} $ with $ Y_{ji} $ but rules out dependence between, say, $ Y_{ij} $ with $ Y_{ik} $. The other two estimators allow for dependence across any two dyads sharing at least one agent in common. The first of these corresponds to the Jackknife estimate of the leading term in the asymptotic variance of $ \sqrt{N}\left(\hat{\theta}-\theta_{0}\right) $. The second estimate is a bias corrected variance estimate which also includes (estimates of) higher order variance terms.

In [6]:
help(netrics.dyadic_regression)

Help on function dyadic_regression in module netrics.dyadic_regression:

dyadic_regression(Y, R, regmodel='normal', directed=True, nocons=False, silent=False, cov='DR_bc')
    AUTHOR: Bryan S. Graham, UC - Berkeley, bgraham@econ.berkeley.edu, July 2017
            (revised September 2018)
    PYTHON 3.6
    
    This function computes dyadic regression estimates under linear, logit or poisson conditional mean
    models. The outcome may be directed or undirected. The outcome model is E[Y_ij|R_ij=r] = r'theta, 
    exp(r'theta)/[1+exp(r'theta)], or exp(r'theta) depending on whether the regression model is chosen to be
    linear, logit or poisson. A variety of standard error estimates, as described further below, are
    reported along with point estimates of theta. Function only works with balanced datasets at the present 
    time. The basic procedure is as described in the _Handbook of Econometrics_ chaper by
    Graham (forthcoming),
        
    N = number of agents
    n = N(N-1),

We begin by attempting to replicate column 6 of Table 3 in Santos Silva and Tenreyro (2006, p. 650). The point estimates reported below agree with the ones reported in their table up to the three decimal places reported (with the exception of a few cases). Note the summary statistics computed using the downloaded replication file differ very slightly from those reported in Table A5 of the paper. This, along with optimization errors, likely account for the small discrepancies.    

The standard errors reported below differ from those in the paper. This is because the paper uses a basic heteroscedastic robust variance-covariance estimate (see p. 645). This variance estimator requires independence of $ Y_{ij} $ and $ Y_{ji} $. In subsequent work Santos Silva and Tenreyro (2010, _Annual Review of Economics_) instead "cluster" on dyads; this corresponds to choosing the 'ind' option for variance estimation in the **dyadic_regression()** command. Of course neither of these two variance estimators make particularly plausible assumptions about the dependence structure of the data.

In [7]:
[theta_DR, vcov_theta_DR]= netrics.dyadic_regression(Y, W, regmodel='poisson', directed=True, nocons=True, \
                                                     silent=False, cov='ind')

/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices


Fisher-Scoring Derivative check (2-norm): 0.10803891
Value of -logL = -15183641.135391,  2-norm of score = 44724805.947635
Value of -logL = -15419066.810454,  2-norm of score = 18265431.975691
Value of -logL = -17556821.955322,  2-norm of score = 23277098.693045
Value of -logL = -17607826.140885,  2-norm of score = 7930537.958928
Value of -logL = -19468215.892431,  2-norm of score = 12731747.589155
Value of -logL = -20885089.422488,  2-norm of score = 96885103.159100
Value of -logL = -21421054.591814,  2-norm of score = 17605387.771348
Value of -logL = -21445918.161899,  2-norm of score = 1726319.684780
Value of -logL = -22293590.325171,  2-norm of score = 9227486.656332
Value of -logL = -22300652.101634,  2-norm of score = 977704.460716
Value of -logL = -22562892.771756,  2-norm of score = 1862539.768510
Value of -logL = -22563241.013345,  2-norm of score = 794371.351692
Value of -logL = -22766084.088668,  2-norm of score = 1448293.573402
Value of -logL = -22766287.436618,  2-norm of 

Next we repeat estimation, but this time compute standard errors using the bias corrected variance estimate mentioned above.

In [8]:
[theta_DR, vcov_theta_DR]= netrics.dyadic_regression(Y, W, regmodel='poisson', directed=True, nocons=True, \
                                                     silent=False, cov='DR_bc')

Fisher-Scoring Derivative check (2-norm): 0.10803891
Value of -logL = -15183641.135391,  2-norm of score = 44724805.947635
Value of -logL = -15419066.810454,  2-norm of score = 18265431.975691
Value of -logL = -17556821.955322,  2-norm of score = 23277098.693045
Value of -logL = -17607826.140885,  2-norm of score = 7930537.958928
Value of -logL = -19468215.892431,  2-norm of score = 12731747.589155
Value of -logL = -20885089.422488,  2-norm of score = 96885103.159100
Value of -logL = -21421054.591814,  2-norm of score = 17605387.771348
Value of -logL = -21445918.161899,  2-norm of score = 1726319.684780
Value of -logL = -22293590.325171,  2-norm of score = 9227486.656332
Value of -logL = -22300652.101634,  2-norm of score = 977704.460716
Value of -logL = -22562892.771756,  2-norm of score = 1862539.768510
Value of -logL = -22563241.013345,  2-norm of score = 794371.351692
Value of -logL = -22766084.088668,  2-norm of score = 1448293.573402
Value of -logL = -22766287.436618,  2-norm of 

For some covariates the increase in the standard error associated with allowing for dependence across dyads with agents in common is very small. But in some cases the standard errors increase quite a bit. Consider *comfrt_wto*, which is a dummy variable equal to 1 if the two trading partners are party to a (common) preferential trading agreement and zero otherwise. The standard error on this coefficient increases by almost 50 percent. Furthermore this is an important trade policy variable.

# Monte Carlo

Next we conduct a short Monte Carlo experiment to illustrate the properties of inference methods based on the different variance-covariance estimates available in **dyad_regression()**. We set $ N = 200 $ and generate outcome data for all $N(N-1)$ ordered pairs of agents according to the outcome model:

$$ Y_{ij}=\exp\left(\theta_{1}W_{ij}+\theta_{2}X_{i}+\theta_{2}X_{j}\right)A_{i}A_{j}U_{ij} $$

Here $A_{i}$, for $i=1,...,N$, is a sequence of iid log normal random variables, each with mean 1 and scale parameter $ \sigma_{A} $; $U_{ij}$ for $i=1,...,n$ with $n=N(N-1)$ is also sequence of iid log normal random variables, each with mean 1 and scale parameter $ \sigma $.    

Each agent is uniformaly at random assigned a location on the unit square, $W_{ij}$ equals the distance between agents *i* and *j* on that square; $X_{i}$ is a standard uniform random variable.    

We set $\theta_{1}=-1$, $\theta_{1}=-1/2$ and $\theta_{3}=1/2$. We set $ \sigma = 1$ and $ \sigma_{A} = 1/4$. This generates moderate, but meaningful, dependence across any two dyads sharing at least one agent in common.   

In [8]:
N = 200
sigmaU = 1
sigmaA = 1/4
eta = [-1, -1/2, 1/2]

B = 1000

theta    = np.zeros((B,3))
coverage = np.zeros((B,9))
se       = np.zeros((B,9))


for b in range(0,B):
    start = time.time()
    
    A = np.random.lognormal(-sigmaA/2, sigmaA, N)
    X1 = np.random.uniform(0, 1, N)
    X2 = np.random.uniform(0, 1, N)
    X3 = np.random.uniform(0, 1, N)

    D = []
    W = []
    for i, j in it.permutations(range(0,N), 2):
        W_ij = ((X1[i] - X1[j])**2 + (X2[i] - X2[j])**2)**(1/2)
        Y_ij = (np.exp(W_ij*eta[0] + X3[i]*eta[1] + X3[j]*eta[2]))*A[i]*A[j]*np.random.lognormal(-sigmaU/2, sigmaU)
        W.append([Y_ij, W_ij, X3[i], X3[j], i, j])
    

    W = pd.DataFrame(W, columns=['Y_ij', 'W_ij', 'X3_i', 'X3_j', 'i', 'j'])    
    W = W.set_index(['i', 'j'], drop = True)  # Set dataframe multi-index
    W['constant'] = 1

    # Get outcome variable and drop from regressor matrix
    Y = W['Y_ij'].copy(deep=True)
    W.drop('Y_ij', axis=1, inplace=True)
    
    # Standard errors based on independence across dyads
    [theta_DR, vcov_theta_DR]= netrics.dyadic_regression(Y, W, regmodel='poisson', directed=True, nocons=True, \
                                                         silent=True, cov='ind')
    
    theta[b,:]    = theta_DR[0:3,:].T
    
    # Coverage
    coverage[b,0] = (eta[0]<=theta_DR[0] + 1.96*np.sqrt(vcov_theta_DR[0,0]))*\
                    (eta[0]>=theta_DR[0] - 1.96*np.sqrt(vcov_theta_DR[0,0]))
    
    coverage[b,1] = (eta[1]<=theta_DR[1] + 1.96*np.sqrt(vcov_theta_DR[1,1]))*\
                    (eta[1]>=theta_DR[1] - 1.96*np.sqrt(vcov_theta_DR[1,1]))
        
    coverage[b,2] = (eta[2]<=theta_DR[2] + 1.96*np.sqrt(vcov_theta_DR[2,2]))*\
                    (eta[2]>=theta_DR[2] - 1.96*np.sqrt(vcov_theta_DR[2,2]))
           
    # Standard error length
    se[b,0]       = np.sqrt(vcov_theta_DR[0,0])
    se[b,1]       = np.sqrt(vcov_theta_DR[1,1])
    se[b,2]       = np.sqrt(vcov_theta_DR[2,2])
    
    
    # Jackknife standard errors
    [theta_DR, vcov_theta_DR]= netrics.dyadic_regression(Y, W, regmodel='poisson', directed=True, nocons=True, \
                                                         silent=True, cov='DR')
    
    coverage[b,3] = (eta[0]<=theta_DR[0] + 1.96*np.sqrt(vcov_theta_DR[0,0]))*\
                    (eta[0]>=theta_DR[0] - 1.96*np.sqrt(vcov_theta_DR[0,0]))
    
    coverage[b,4] = (eta[1]<=theta_DR[1] + 1.96*np.sqrt(vcov_theta_DR[1,1]))*\
                    (eta[1]>=theta_DR[1] - 1.96*np.sqrt(vcov_theta_DR[1,1]))
        
    coverage[b,5] = (eta[2]<=theta_DR[2] + 1.96*np.sqrt(vcov_theta_DR[2,2]))*\
                    (eta[2]>=theta_DR[2] - 1.96*np.sqrt(vcov_theta_DR[2,2]))
    
    # Standard error length
    se[b,3]       = np.sqrt(vcov_theta_DR[0,0])
    se[b,4]       = np.sqrt(vcov_theta_DR[1,1])
    se[b,5]       = np.sqrt(vcov_theta_DR[2,2])
    
    # Bias corrected standard errors
    [theta_DR, vcov_theta_DR]= netrics.dyadic_regression(Y, W, regmodel='poisson', directed=True, nocons=True, \
                                                         silent=True, cov='DR_bc')
    
    coverage[b,6] = (eta[0]<=theta_DR[0] + 1.96*np.sqrt(vcov_theta_DR[0,0]))*\
                    (eta[0]>=theta_DR[0] - 1.96*np.sqrt(vcov_theta_DR[0,0]))
    
    coverage[b,7] = (eta[1]<=theta_DR[1] + 1.96*np.sqrt(vcov_theta_DR[1,1]))*\
                    (eta[1]>=theta_DR[1] - 1.96*np.sqrt(vcov_theta_DR[1,1]))
        
    coverage[b,8] = (eta[2]<=theta_DR[2] + 1.96*np.sqrt(vcov_theta_DR[2,2]))*\
                    (eta[2]>=theta_DR[2] - 1.96*np.sqrt(vcov_theta_DR[2,2]))
    
    # Standard error length
    se[b,6]       = np.sqrt(vcov_theta_DR[0,0])
    se[b,7]       = np.sqrt(vcov_theta_DR[1,1])
    se[b,8]       = np.sqrt(vcov_theta_DR[2,2])
        
    end = time.time()
    if (b+1) % 100 == 0:
        print("Time required f/ MC rep  " + str(b+1) + " of " + str(B) + ": " + str(end-start))      


/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

Time required f/ MC rep  100 of 1000: 1.7452309131622314


/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

...

/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

Time required f/ MC rep  900 of 1000: 1.180293083190918


/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraham/Dropbox/Sites/software/netrics/netrics/dyadic_regression.py:113: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  agents  = set(i | j)            # Set of all unique agent-level indices
/Users/bgraha

Time required f/ MC rep  1000 of 1000: 1.2227239608764648


The following table reports Monte Carlo coverage estimates for Wald type nominal 95 percent confidence intervals based on the three variance estimates available in **dyadic_regression()**. The first assumes independence across dyads ('ind'), the second is the jackknife estimate ('DR'), and the third a bias corrected estimate ('DR_bc'). These last two allow for dependence across dyads sharing at least one agent in common. The columns in the table correspond to the three parameters in the model, the rows to the three variance estimators.   

The table shows that assuming independence across dyads results in considerable undercoverage. The jackknife estimate, as would be expected, is conservative. Finally the biased corrected estimate has actual coverage very close to nominal coverage.   

The standard error associated with a Monte Carlo coverage estimate is $\sqrt{\alpha\left(1-\alpha\right)/B}$. With $B = 1,000$ simulations and $\alpha = 0.05$ this results in a standard error of approximately 0.007 or a 95 percent confidence interval of $[0.936, 0.964]$. Hence the 'ind' standard errors significantly undercover, while the 'DR' standard errors (sometimes) significantly overcover. The 'DR_bc' have actual coverage (almost) insignificantly different from target coverage.

In [9]:
print(np.mean(coverage,axis=0).reshape((3,3)))

[[0.775 0.538 0.55 ]
 [0.97  0.954 0.951]
 [0.928 0.941 0.932]]


Next we look at average standard error length. The 'DR_bc' standard errors, while appreciably large than those base bone the (incorrect) assumption of independence across dyads, are smaller than the jackknife estimates.

In [10]:
print(np.mean(se,axis=0).reshape((3,3)))

[[0.03183488 0.02629007 0.02628867]
 [0.05963431 0.07108533 0.07087968]
 [0.05034701 0.06601465 0.06579389]]


In [11]:
# This imports an attractive notebook style from Github
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen('http://bit.ly/1Bf5Hft')
HTML(html.read().decode('utf-8'))